In [1]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
def cache_replc_lru(n=5, m=2):
    """
    Create a cache model with LRU replacement.
    
    Args:
        n: Number of items in the universe
        m: Cache capacity (number of items that can be stored)
    """
    model = Network('model')
    
    # Block 1: nodes
    delay = Delay(model, 'Delay')
    cache_node = Cache(model, 'Cache', n, m, ReplacementStrategy.LRU)
    
    # Block 2: classes
    job_class = ClosedClass(model, 'JobClass', 1, delay, 0)
    hit_class = ClosedClass(model, 'HitClass', 0, delay, 0)
    miss_class = ClosedClass(model, 'MissClass', 0, delay, 0)
    
    # Set service time at delay node
    delay.set_service(job_class, Exp(1))
    
    # Set cache access pattern (uniform over items)
    p_access = DiscreteSampler([1/n] * n)
    cache_node.set_read(job_class, p_access)
    
    # Set hit and miss classes
    cache_node.set_hit_class(job_class, hit_class)
    cache_node.set_miss_class(job_class, miss_class)
    
    # Block 3: topology
    P = model.init_routing_matrix()
    P.add_route(job_class, delay, cache_node, 1.0)
    P.add_class_switch(hit_class, job_class, cache_node, delay, 1.0)
    P.add_class_switch(miss_class, job_class, cache_node, delay, 1.0)
    model.link(P)
    
    return model

# Create the model
model = cache_replc_lru(n=5, m=2)

In [3]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# CTMC Solver
solver_ctmc = CTMC(model, keep=False)
avg_node_table_ctmc = solver_ctmc.avg_node_table()
print("\nCTMC Solver:")
print(avg_node_table_ctmc)

# MVA Solver
model.reset()
solver_mva = MVA(model)
avg_node_table_mva = solver_mva.avg_node_table()
print("\nMVA Solver:")
print(avg_node_table_mva)


=== Solver Results ===
                Node   JobClass  QLen  Util  RespT  ResidT  ArvR  Tput
0              Delay   JobClass   1.0   1.0    1.0     1.0   1.0   1.0
3              Cache   JobClass   0.0   0.0    0.0     0.0   1.0   0.0
4              Cache   HitClass   0.0   0.0    0.0     0.0   0.0   0.4
5              Cache  MissClass   0.0   0.0    0.0     0.0   0.0   0.6
6  CS_Cache_to_Delay   JobClass   0.0   0.0    0.0     0.0   0.0   1.0
7  CS_Cache_to_Delay   HitClass   0.0   0.0    0.0     0.0   0.4   0.0
8  CS_Cache_to_Delay  MissClass   0.0   0.0    0.0     0.0   0.6   0.0

CTMC Solver:
                Node   JobClass  QLen  Util  RespT  ResidT  ArvR  Tput
0              Delay   JobClass   1.0   1.0    1.0     1.0   1.0   1.0
3              Cache   JobClass   0.0   0.0    0.0     0.0   1.0   0.0
4              Cache   HitClass   0.0   0.0    0.0     0.0   0.0   0.4
5              Cache  MissClass   0.0   0.0    0.0     0.0   0.0   0.6
6  CS_Cache_to_Delay   JobClass   0.0  